In [1]:
%load_ext autoreload
%autoreload 2
from datetime import datetime
import time
import io
import os
import argparse

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tqdm

import tensorflow as tf
import tensorflow_probability as tfp
tf.keras.backend.set_floatx('float32')

import data_utils
import gan_utils
import gan

# os.environ["OMP_NUM_THREADS"] = "4"
# os.environ["OPENBLAS_NUM_THREADS"] = "4"
# os.environ["MKL_NUM_THREADS"] = "4"
# os.environ["VECLIB_MAXIMUM_THREADS"] = "4"
# os.environ["NUMEXPR_NUM_THREADS"] = "4"
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
n_iters = 100
batch_size = 64
sinkhorn_eps = 100 # entropy regularisation coefficent (will take log10 of this number and round to int)
sinkhorn_l = 100 # number of sinkhorn iterations
reg_penalty = 100 # martingale regularisation penalty (will take log10 of this number and round to int)
lr = 1e-3

gen_type = 'lstmd'
activation = 'tanh'
nlstm = 1
g_state_size = 64
d_state_size = 64
log_series = True

dname = 'SPX'
seq_dim = 1 # dimension of the time series excluding time dimension
Dx = 2 # dimension of the time series including time dimension
time_steps = 250
sample_len = 300
hist_len = 50
stride = 50
seed = 42 # np.random.randint(0, 10000)
dt = 1 / 252

In [3]:
training_params = {
    'n_iters': n_iters,
    'batch_size': batch_size,
    'sinkhorn_eps': sinkhorn_eps,
    'sinkhorn_l': sinkhorn_l,
    'reg_penalty': reg_penalty,
    'lr': lr,
}

model_params = {
    'gen_type': gen_type,
    'activation': activation,
    'nlstm': nlstm,
    'g_state_size': g_state_size,
    'd_state_size': d_state_size,
    'log_series': log_series,
}

data_params = {
    'dname': dname,
    'dt': dt,
    'time_steps': time_steps,
    'seed': seed,
    'Dx': Dx,
}

In [4]:
parser = argparse.ArgumentParser(description='cot')

# parser.add_argument('-d', '--dname', type=str, default='GBM',
#                     choices=['SineImage', 'AROne', 'eeg', 'GBM'])
parser.add_argument('-t', '--test', type=str, default='cot',
                    choices=['cot'])
# parser.add_argument('-s', '--seed', type=int, default=42)
# parser.add_argument('-gss', '--g_state_size', type=int, default=32)
# parser.add_argument('-dss', '--d_state_size', type=int, default=32)
parser.add_argument('-gfs', '--g_filter_size', type=int, default=32)
parser.add_argument('-dfs', '--d_filter_size', type=int, default=32)
# parser.add_argument('-r', '--reg_penalty', type=float, default=10.0) # martingale regularisation coefficent
# parser.add_argument('-ts', '--time_steps', type=int, default=60)
# parser.add_argument('-sinke', '--sinkhorn_eps', type=float, default=100) # entropy regularisation coefficent
# parser.add_argument('-sinkl', '--sinkhorn_l', type=int, default=100) # number of sinkhorn iterations
# parser.add_argument('-Dx', '--Dx', type=int, default=1)
parser.add_argument('-Dy', '--Dy', type=int, default=10)
parser.add_argument('-Dz', '--z_dims_t', type=int, default=4)
# parser.add_argument('-g', '--gen', type=str, default="genlstm",
#                     choices=["lstm", "fc", "genlstm"])
# parser.add_argument('-bs', '--batch_size', type=int, default=64)
# parser.add_argument('-nlstm', '--nlstm', type=int, default=1,
                    # help="number of lstms in discriminator")
# parser.add_argument('-lr', '--lr', type=float, default=1e-3)
parser.add_argument('-bn', '--bn', type=int, default=1,
                    help="batch norm")

args, unknown = parser.parse_known_args()

In [5]:
start_time = time.time()
# hyper-parameter settings
# dname = args.dname
test = args.test
# time_steps = args.time_steps
# batch_size = args.batch_size
bn = bool(args.bn)
# if "SLURM_ARRAY_TASK_ID" in os.environ:
#     seed = int(os.environ["SLURM_ARRAY_TASK_ID"])
# else:
#     seed = args.seed

# Dx = args.Dx
g_output_activation = 'linear'

df = pd.read_csv('./data/spx_20231229.csv', index_col=0, parse_dates=True)
if dname == 'AROne':
    data_dist = data_utils.AROne(
        Dx, time_steps, np.linspace(0.1, 0.9, Dx), 0.5)
elif dname == 'eeg':
    data_dist = data_utils.EEGData(
        Dx, time_steps, batch_size, n_iters, seed=seed)
elif dname == 'SineImage':
    data_dist = data_utils.SineImage(
        length=time_steps, Dx=Dx, rand_std=0.1)
elif dname == 'GBM':
    data_dist = data_utils.GBM(mu=0.2, sigma=0.5, dt=dt, length=time_steps, batch_size=batch_size, n_paths=batch_size*100,
                               log_series=log_series, initial_value=1.0, time_dim=False, seed=seed)
elif dname == 'OU':
    data_dist = data_utils.OU(kappa=10., theta=1., sigma=0.5, dt=dt, length=time_steps, batch_size=batch_size, n_paths=batch_size*100,
                              log_series=log_series, initial_value=1.0, time_dim=False, seed=seed)
elif dname == 'Heston':
    data_dist = data_utils.Heston(mu=0.2, v0=0.25, kappa=1., theta=0.16, rho=-0.7, sigma=0.2, dt=dt, length=time_steps, batch_size=batch_size, n_paths=batch_size*100,
                                  log_series=log_series, initial_value=1.0, time_dim=False, seed=seed)
elif dname == 'SPX':
    data_dist = data_utils.DFDataset(df, '1995-01-01', '2022-10-19', sample_len, batch_size, stride)
else:
    ValueError('Data does not exist.')

dataset = dname
# Number of RNN layers stacked together
n_layers = 1
# reg_penalty = args.reg_penalty
# gen_lr = args.lr
# disc_lr = args.lr
gen_lr = lr
disc_lr = lr
tf.random.set_seed(seed)
np.random.seed(seed)
# Add gradient clipping before updates
gen_optimiser = tf.keras.optimizers.legacy.Adam(gen_lr)
dischm_optimiser = tf.keras.optimizers.legacy.Adam(disc_lr)

it_counts = 0
disc_iters = 1
# sinkhorn_eps = args.sinkhorn_eps
# sinkhorn_l = args.sinkhorn_l
# nlstm = args.nlstm
scaling_coef = 1.0

# Define a standard multivariate normal for
# (z1, z2, ..., zT) --> (y1, y2, ..., yT)
z_dims_t = args.z_dims_t
y_dims = args.Dy
# dist_z = tfp.distributions.Uniform(-1, 1)
# dist_z = tfp.distributions.Normal(0, 1)
dist_z = data_utils.GARCH(df, start_date='1995-01-01', end_date='2022-10-19', sample_len=300,
                          p=20, o=0, q=0, mean_model='Zero', vol_model='GARCH', dist='gaussian',
                          seed=42, stride=50)
# dist_y = tfp.distributions.Uniform(-1, 1)

# Create instances of generator, discriminator_h and
# discriminator_m CONV VERSION
# g_state_size = args.g_state_size
# d_state_size = args.d_state_size
g_filter_size = args.g_filter_size
d_filter_size = args.d_filter_size
disc_kernel_width = 5

if gen_type == "fc":
    generator = gan.SimpleGenerator(
        batch_size, time_steps, Dx, g_filter_size, z_dims_t,
        output_activation=g_output_activation)
elif gen_type == "lstm":
    generator = gan.ToyGenerator(
        batch_size, time_steps, z_dims_t, Dx, g_state_size, g_filter_size,
        output_activation=g_output_activation, nlstm=nlstm, nlayer=2,
        Dy=y_dims, bn=bn)
elif gen_type == "genlstm":
    generator = gan.GenLSTM(z_dims_t, Dx, time_steps, hidden_size=g_state_size, activation=activation, n_lstm_layers=nlstm, log_series=log_series)
elif gen_type == "lstmp":
    generator = gan.GenLSTMp(z_dims_t, Dx, time_steps, hidden_size=g_state_size, activation=activation, n_lstm_layers=nlstm, log_series=log_series)
elif gen_type == "lstmpdt":
    generator = gan.GenLSTMpdt(z_dims_t, Dx, time_steps, dt, hidden_size=g_state_size, activation=activation, n_lstm_layers=nlstm, log_series=log_series)
elif gen_type == "lstmd":
    generator = gan.GenLSTMd(z_dims_t, seq_dim, sample_len, hist_len, hidden_size=g_state_size)

discriminator_h = gan.ToyDiscriminator(
    batch_size, time_steps, z_dims_t, Dx, d_state_size, d_filter_size,
    kernel_size=disc_kernel_width, nlayer=2, nlstm=0, bn=bn)
discriminator_m = gan.ToyDiscriminator(
    batch_size, time_steps, z_dims_t, Dx, d_state_size, d_filter_size,
    kernel_size=disc_kernel_width, nlayer=2, nlstm=0, bn=bn)

# data_utils.check_model_summary(batch_size, z_dims, generator)
# data_utils.check_model_summary(batch_size, seq_len, discriminator_h)

lsinke = int(np.round(np.log10(sinkhorn_eps)))
lreg = int(np.round(np.log10(reg_penalty)))
saved_file = f"{dname[:3]}_{test[0]}_e{lsinke:d}r{lreg:d}s{seed:d}" + \
    "{}_{}{}-{}-{}-{}-{}".format(dataset, datetime.now().strftime("%h"),
                                    datetime.now().strftime("%d"),
                                    datetime.now().strftime("%H"),
                                    datetime.now().strftime("%M"),
                                    datetime.now().strftime("%S"),
                                    datetime.now().strftime("%f"))

model_fn = "%s_Dz%d_Dy%d_Dx%d_bs%d_gss%d_gfs%d_dss%d_dfs%d_ts%d_r%d_eps%d_l%d_lr%d_nl%d_s%02d" % (
    dname, z_dims_t, y_dims, Dx, batch_size, g_state_size, g_filter_size,
    d_state_size, d_filter_size, time_steps, np.round(np.log10(reg_penalty)),
    np.round(np.log10(sinkhorn_eps)), sinkhorn_l, np.round(np.log10(lr)), nlstm, seed)

log_dir = f"./trained/{saved_file}/log"

# Create directories for storing images later.
if not os.path.exists(f"trained/{saved_file}/data"):
    os.makedirs(f"trained/{saved_file}/data")
if not os.path.exists(f"trained/{saved_file}/images"):
    os.makedirs(f"trained/{saved_file}/images")

# GAN train notes
with open("./trained/{}/train_notes.txt".format(saved_file), 'w') as f:
    # Include any experiment notes here:
    f.write("Experiment notes: .... \n\n")
    f.write("MODEL_DATA: {}\nSEQ_LEN: {}\n".format(
        dataset,
        time_steps, ))
    f.write("STATE_SIZE: {}\nNUM_LAYERS: {}\nLAMBDA: {}\n".format(
        g_state_size,
        n_layers,
        reg_penalty))
    f.write("BATCH_SIZE: {}\nCRITIC_ITERS: {}\nGenerator LR: {}\nDiscriminator LR:{}\n".format(
        batch_size,
        disc_iters,
        gen_lr,
        disc_lr))
    f.write("SINKHORN EPS: {}\nSINKHORN L: {}\n\n".format(
        sinkhorn_eps,
        sinkhorn_l))

train_writer = tf.summary.create_file_writer(logdir=log_dir)

with train_writer.as_default():
    tf.summary.text('training_params', data_utils.pretty_json(training_params), step=0)
    tf.summary.text('model_params', data_utils.pretty_json(model_params), step=0)
    tf.summary.text('data_params', data_utils.pretty_json(data_params), step=0)

@tf.function
def disc_training_step(real_data, real_data_p):
    hidden_z = dist_z.sample([batch_size, time_steps, z_dims_t])
    hidden_z_p = dist_z.sample([batch_size, time_steps, z_dims_t])
    # hidden_y = dist_y.sample([batch_size, y_dims])
    # hidden_y_p = dist_y.sample([batch_size, y_dims])

    with tf.GradientTape(persistent=True) as disc_tape:
        # fake_data = generator.call(hidden_z, hidden_y)
        # fake_data_p = generator.call(hidden_z_p, hidden_y_p)
        # fake_data = generator.call(hidden_z)                  # For GBM, OU, Heston
        # fake_data_p = generator.call(hidden_z_p)              # For GBM, OU, Heston
        fake_data = generator.call(hidden_z, real_data)         # For SPX
        fake_data_p = generator.call(hidden_z_p, real_data_p)   # For SPX

        # h_fake = discriminator_h.call(fake_data)
        h_fake = discriminator_h.call(fake_data[:,hist_len:,:]) # For SPX

        # m_real = discriminator_m.call(real_data)
        m_real = discriminator_m.call(real_data[:,hist_len:,:]) # For SPX
        # m_fake = discriminator_m.call(fake_data)
        m_fake = discriminator_m.call(fake_data[:,hist_len:,:]) # For SPX

        # h_real_p = discriminator_h.call(real_data_p)
        h_real_p = discriminator_h.call(real_data_p[:,hist_len:,:]) # For SPX
        # h_fake_p = discriminator_h.call(fake_data_p)
        h_fake_p = discriminator_h.call(fake_data_p[:,hist_len:,:]) # For SPX

        # m_real_p = discriminator_m.call(real_data_p)
        m_real_p = discriminator_m.call(real_data_p[:,hist_len:,:]) # For SPX

        # loss1 = gan_utils.compute_mixed_sinkhorn_loss(
        #     real_data, fake_data, m_real, m_fake, h_fake, scaling_coef,
        #     sinkhorn_eps, sinkhorn_l, real_data_p, fake_data_p, m_real_p,
        #     h_real_p, h_fake_p)
        # print(f'fake_data shape: {fake_data[:,hist_len:,:].shape}')
        # print(f'fake_data_p shape: {fake_data_p[:,hist_len:,:].shape}')
        # print(f'real_data shape: {real_data[:,hist_len:,:].shape}')
        # print(f'real_data_p shape: {real_data_p[:,hist_len:,:].shape}')
        # print(f'm_real shape: {m_real.shape}')
        # print(f'm_fake shape: {m_fake.shape}')
        # print(f'h_fake shape: {h_fake.shape}')
        # print(f'm_real_p shape: {m_real_p.shape}')
        # print(f'h_real_p shape: {h_real_p.shape}')
        # print(f'h_fake_p shape: {h_fake_p.shape}')
        loss1 = gan_utils.compute_mixed_sinkhorn_loss(
            real_data[:,hist_len:,:], fake_data[:,hist_len:,:], m_real, m_fake, h_fake, scaling_coef,
            sinkhorn_eps, sinkhorn_l, real_data_p[:,hist_len:,:], fake_data_p[:,hist_len:,:], m_real_p,
            h_real_p, h_fake_p)
        pm1 = gan_utils.scale_invariante_martingale_regularization(
            m_real, reg_penalty, scaling_coef)
        disc_loss = - loss1 + pm1
    # update discriminator parameters
    disch_grads, discm_grads = disc_tape.gradient(
        disc_loss, [discriminator_h.trainable_variables, discriminator_m.trainable_variables])
    dischm_optimiser.apply_gradients(zip(disch_grads, discriminator_h.trainable_variables))
    dischm_optimiser.apply_gradients(zip(discm_grads, discriminator_m.trainable_variables))

@tf.function
def gen_training_step(real_data, real_data_p):
    hidden_z = dist_z.sample([batch_size, time_steps, z_dims_t])
    hidden_z_p = dist_z.sample([batch_size, time_steps, z_dims_t])
    # hidden_y = dist_y.sample([batch_size, y_dims])
    # hidden_y_p = dist_y.sample([batch_size, y_dims])

    with tf.GradientTape() as gen_tape:
        # fake_data = generator.call(hidden_z, hidden_y)
        # fake_data_p = generator.call(hidden_z_p, hidden_y_p)
        # fake_data = generator.call(hidden_z)                      # For GBM, OU, Heston
        # fake_data_p = generator.call(hidden_z_p)                  # For GBM, OU, Heston
        fake_data = generator.call(hidden_z, real_data)             # For SPX
        fake_data_p = generator.call(hidden_z_p, real_data_p)       # For SPX

        # h and m networks used to compute the martingale penalty

        # h_fake = discriminator_h.call(fake_data)
        h_fake = discriminator_h.call(fake_data[:,hist_len:,:]) # For SPX

        # m_real = discriminator_m.call(real_data)
        m_real = discriminator_m.call(real_data[:,hist_len:,:]) # For SPX
        # m_fake = discriminator_m.call(fake_data)
        m_fake = discriminator_m.call(fake_data[:,hist_len:,:]) # For SPX

        # h_real_p = discriminator_h.call(real_data_p)
        h_real_p = discriminator_h.call(real_data_p[:,hist_len:,:]) # For SPX
        # h_fake_p = discriminator_h.call(fake_data_p)
        h_fake_p = discriminator_h.call(fake_data_p[:,hist_len:,:]) # For SPX

        # m_real_p = discriminator_m.call(real_data_p)
        m_real_p = discriminator_m.call(real_data_p[:,hist_len:,:]) # For SPX

        # loss2 = gan_utils.compute_mixed_sinkhorn_loss(
        #     real_data, fake_data, m_real, m_fake, h_fake, scaling_coef,
        #     sinkhorn_eps, sinkhorn_l, real_data_p, fake_data_p, m_real_p,
        #     h_real_p, h_fake_p)
        loss2 = gan_utils.compute_mixed_sinkhorn_loss(
            real_data[:,hist_len:,:], fake_data[:,hist_len:,:], m_real, m_fake, h_fake, scaling_coef,
            sinkhorn_eps, sinkhorn_l, real_data_p[:,hist_len:,:], fake_data_p[:,hist_len:,:], m_real_p,
            h_real_p, h_fake_p)
        gen_loss = loss2
    # update generator parameters
    generator_grads = gen_tape.gradient(
        gen_loss, generator.trainable_variables)
    gen_optimiser.apply_gradients(zip(generator_grads, generator.trainable_variables))
    return loss2

with tqdm.trange(n_iters, ncols=150) as it:
    for _ in it:
        it_counts += 1
        # generate a batch of REAL data
        real_data = data_dist.batch(batch_size)
        real_data_p = data_dist.batch(batch_size)
        real_data = tf.cast(real_data, tf.float32)
        real_data_p = tf.cast(real_data_p, tf.float32)

        disc_training_step(real_data, real_data_p)
        loss = gen_training_step(real_data, real_data_p)
        it.set_postfix(loss=float(loss))

        with train_writer.as_default():
            tf.summary.scalar('Sinkhorn loss', loss, step=it_counts)
            train_writer.flush()

        if not np.isfinite(loss.numpy()):
            print('%s Loss exploded!' % model_fn)
            # Open the existing file with mode a - append
            with open("./trained/{}/train_notes.txt".format(saved_file), 'a') as f:
                # Include any experiment notes here:
                f.write("\n Training failed! ")
            break
        else:
            # print("Plot samples produced by generator after %d iterations" % it_counts)
            z = dist_z.sample([batch_size, time_steps, z_dims_t])
            # y = dist_y.sample([batch_size, y_dims])
            # samples = generator.call(z, y, training=False)
            samples = generator.call(z, training=False)
            batch_series = np.asarray(samples[..., 1:])
            if log_series:
                plt.plot(np.exp(batch_series.T))
                sample_mean = np.diff(batch_series, axis=1).mean() / dt
                sample_std = np.diff(batch_series, axis=1).std() / np.sqrt(dt)
            else:
                plt.plot(batch_series.T)
                sample_mean = np.diff(np.log(batch_series), axis=1).mean() / dt
                sample_std = np.diff(np.log(batch_series), axis=1).std() / np.sqrt(dt)
            # save plot to file
            # if samples.shape[-1] == 1:
            #     data_utils.plot_batch(np.asarray(samples[..., 0]), it_counts, saved_file)

            # img = tf.transpose(tf.concat(list(samples), axis=1))[None, :, :, None]
            with train_writer.as_default():
                tf.summary.image("Generated samples", data_utils.plot_to_image(plt.gcf()), step=it_counts)
                tf.summary.scalar('Stats/Sample_mean', sample_mean, step=it_counts)
                tf.summary.scalar('Stats/Sample_std', sample_std, step=it_counts)
            # save model to file
            generator.save_weights(f"./trained/{saved_file}/generator/")
            discriminator_h.save_weights(f"./trained/{saved_file}/discriminator_h/")
            discriminator_m.save_weights(f"./trained/{saved_file}/discriminator_m/")
        continue

print("--- The entire training takes %s minutes ---" % ((time.time() - start_time) / 60.0))

Optimization terminated successfully    (Exit mode 0)
            Current function value: 5855.988337142852
            Iterations: 37
            Function evaluations: 859
            Gradient evaluations: 37
                        Zero Mean - ARCH Model Results                        
Dep. Variable:           gaussianized   R-squared:                       0.000
Mean Model:                 Zero Mean   Adj. R-squared:                  0.000
Vol Model:                       ARCH   Log-Likelihood:               -5855.99
Distribution:                  Normal   AIC:                           11754.0
Method:            Maximum Likelihood   BIC:                           11897.9
                                        No. Observations:                 6999
Date:                Tue, Apr 02 2024   Df Residuals:                     6999
Time:                        17:40:10   Df Model:                            0
                               Volatility Model                              
 

2024-04-02 17:40:10.374262: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-04-02 17:40:10.374285: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-04-02 17:40:10.374294: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-04-02 17:40:10.374325: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-04-02 17:40:10.374339: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
  0%|                                                                                                                         | 0/100 [00:00<?, ?it/s]

fake_data shape: (64, 250, 2)
fake_data_p shape: (64, 250, 2)
real_data shape: (64, 250, 2)
real_data_p shape: (64, 250, 2)
m_real shape: (64, 250, 64)
m_fake shape: (64, 250, 64)
h_fake shape: (64, 250, 64)
m_real_p shape: (64, 250, 64)
h_real_p shape: (64, 250, 64)
h_fake_p shape: (64, 250, 64)
fake_data shape: (64, 250, 2)
fake_data_p shape: (64, 250, 2)
real_data shape: (64, 250, 2)
real_data_p shape: (64, 250, 2)
m_real shape: (64, 250, 64)
m_fake shape: (64, 250, 64)
h_fake shape: (64, 250, 64)
m_real_p shape: (64, 250, 64)
h_real_p shape: (64, 250, 64)
h_fake_p shape: (64, 250, 64)


2024-04-02 17:44:32.908860: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
